### IMPORTAR LIRERÍAS Y LEER DATOS

In [ ]:
import numpy as np
import pandas as pd

address = 'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD/Basededatos/RDEV.xlsx'
address2 = 'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD/Basededatos/RDEV.xlsx'
address3 = 'C:\\Users\\mticllacu\\OneDrive - LUZ DEL SUR S.A.A\\Archivos de Ronald Quispe Ocaña - ProyectoRyD\\Basededatos\\RDEV.xlsx'
#df = pd.read_excel(address, header=1)
df = pd.read_excel(address, header=1)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoR&D/Basededatos/RDEV.xlsx'

In [ ]:
# df = df.dropna(axis=1,how = 'all')
df = df.iloc[:,1:]
df['FECHA']= pd.to_datetime(df['FECHA'])
df.head(10)

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS24 T 75°C [mΩ],H POS25 T 75°C [mΩ],H POS26 T 75°C [mΩ],H POS27 T 75°C [mΩ],X POS1 R 75°C[mΩ],X POS1 S 75°C[mΩ],X POS1 T 75°C[mΩ],Y POS1 R 75°C[mΩ],Y POS1 S 75°C[mΩ],Y POS1 T 75°C[mΩ]
0,146916,2019-06-30,149.785000,147.781000,144.857000,142.047000,140.025000,137.230000,134.387000,132.398000,...,143.481000,145.522000,148.221000,151.179000,5.348000,5.343000,5.334000,NaN,NaN,NaN
1,146916,2015-01-06,148.965511,147.057225,144.194798,141.451638,139.424085,136.800193,133.937765,132.148748,...,143.598459,146.222351,148.846243,151.708671,11.319709,11.180166,11.883846,NaN,NaN,NaN
2,146916,2014-11-12,144.028680,142.225767,139.389184,136.732893,134.761709,132.081379,129.340951,127.417845,...,138.403592,140.338718,142.995010,145.807553,5.170520,5.183673,5.190792,NaN,NaN,NaN
3,146917,2022-05-12,153.255000,150.592000,147.916000,145.976000,143.261000,140.604000,138.741000,136.053000,...,147.560000,150.389000,152.156000,155.001000,14.540000,14.560000,14.730000,NaN,NaN,NaN
4,146917,2014-12-05,157.346194,154.665864,151.913417,149.942233,147.153728,144.449359,142.514233,139.785825,...,151.516777,154.389417,156.216369,159.125068,12.418000,12.455000,125.590000,NaN,NaN,NaN
5,146918,2019-07-11,138.333000,136.501000,133.945000,131.398000,129.652000,127.065000,124.519000,122.704000,...,132.949000,134.800000,137.290000,140.009000,4.962000,4.960000,4.968000,NaN,NaN,NaN
6,146918,2014-11-25,142.987812,141.265067,138.568081,135.906737,134.112706,131.475125,128.790019,126.924702,...,137.570077,139.494798,142.084856,144.948177,5.131284,5.145423,5.146848,NaN,NaN,NaN


In [ ]:
# Columnas de resistencia (terminan en mΩ)
res_cols = [c for c in df.columns if c.endswith("[mΩ]")]

for col in res_cols:
    # valor inicial por SERIE en la fecha mínima
    ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df["SERIE"].map(ref)

    # variación porcentual
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100

    # reemplazar inf por NaN (división por cero)
    delta = delta.replace([np.inf, -np.inf], np.nan)

    # si alguno es NaN o ref/valor es 0 -> NaN
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan

    # agregar columna nueva
    df[f"{col}_Delta"] = delta

C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\315815301.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\315815301.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\

In [ ]:
df.head()

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS24 T 75°C [mΩ]_Delta,H POS25 T 75°C [mΩ]_Delta,H POS26 T 75°C [mΩ]_Delta,H POS27 T 75°C [mΩ]_Delta,X POS1 R 75°C[mΩ]_Delta,X POS1 S 75°C[mΩ]_Delta,X POS1 T 75°C[mΩ]_Delta,Y POS1 R 75°C[mΩ]_Delta,Y POS1 S 75°C[mΩ]_Delta,Y POS1 T 75°C[mΩ]_Delta
0,146916,2019-06-30,149.785000,147.781000,144.857000,142.047000,140.025000,137.230000,134.387000,132.398000,...,3.668552,3.693408,3.654666,3.683929,3.432531,3.073641,2.758893,NaN,NaN,NaN
1,146916,2015-01-06,148.965511,147.057225,144.194798,141.451638,139.424085,136.800193,133.937765,132.148748,...,3.753419,4.192451,4.091914,4.047196,118.927854,115.680402,128.940916,NaN,NaN,NaN
2,146916,2014-11-12,144.028680,142.225767,139.389184,136.732893,134.761709,132.081379,129.340951,127.417845,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,146917,2022-05-12,153.255000,150.592000,147.916000,145.976000,143.261000,140.604000,138.741000,136.053000,...,2.611445,2.591122,2.599196,2.591715,17.088098,16.900843,88.271359,NaN,NaN,NaN
4,146917,2014-12-05,157.346194,154.665864,151.913417,149.942233,147.153728,144.449359,142.514233,139.785825,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [ ]:
# Identificar las columnas que terminan en "_Delta"
delta_cols = [c for c in df.columns if c.endswith("_Delta")]

# Crear nuevas columnas de puntaje según la regla
for col in delta_cols:
    puntaje_col = col.replace("_Delta", "_Score")  # ejemplo: H POS21..._Delta → H POS21..._Score
    df[puntaje_col] = df[col].apply(
        lambda x: np.nan if pd.isna(x) else (1 if x < 5 else 5) 
    )


C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\1961661274.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[puntaje_col] = df[col].apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\1961661274.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[puntaje_col] = df[col].apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\1961661274.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

In [ ]:
df.head()

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS24 T 75°C [mΩ]_Score,H POS25 T 75°C [mΩ]_Score,H POS26 T 75°C [mΩ]_Score,H POS27 T 75°C [mΩ]_Score,X POS1 R 75°C[mΩ]_Score,X POS1 S 75°C[mΩ]_Score,X POS1 T 75°C[mΩ]_Score,Y POS1 R 75°C[mΩ]_Score,Y POS1 S 75°C[mΩ]_Score,Y POS1 T 75°C[mΩ]_Score
0,146916,2019-06-30,149.785000,147.781000,144.857000,142.047000,140.025000,137.230000,134.387000,132.398000,...,1,1,1,1,1,1,1,NaN,NaN,NaN
1,146916,2015-01-06,148.965511,147.057225,144.194798,141.451638,139.424085,136.800193,133.937765,132.148748,...,1,1,1,1,5,5,5,NaN,NaN,NaN
2,146916,2014-11-12,144.028680,142.225767,139.389184,136.732893,134.761709,132.081379,129.340951,127.417845,...,1,1,1,1,1,1,1,NaN,NaN,NaN
3,146917,2022-05-12,153.255000,150.592000,147.916000,145.976000,143.261000,140.604000,138.741000,136.053000,...,1,1,1,1,5,5,5,NaN,NaN,NaN
4,146917,2014-12-05,157.346194,154.665864,151.913417,149.942233,147.153728,144.449359,142.514233,139.785825,...,1,1,1,1,1,1,1,NaN,NaN,NaN


In [ ]:
# Identificar las columnas que terminan en "_Delta"
delta_cols = [c for c in df.columns if c.endswith("_Score")]

# Crear una nueva columna con el máximo por fila
df["ROHM"] = df[delta_cols].max(axis=1)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_14188\4100089174.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ROHM"] = df[delta_cols].max(axis=1)


In [ ]:
df.head()

,SERIE,FECHA,H POS1 R 75°C [mΩ],H POS2 R 75°C [mΩ],H POS3 R 75°C [mΩ],H POS4 R 75°C [mΩ],H POS5 R 75°C [mΩ],H POS6 R 75°C [mΩ],H POS7 R 75°C [mΩ],H POS8 R 75°C [mΩ],...,H POS25 T 75°C [mΩ]_Score,H POS26 T 75°C [mΩ]_Score,H POS27 T 75°C [mΩ]_Score,X POS1 R 75°C[mΩ]_Score,X POS1 S 75°C[mΩ]_Score,X POS1 T 75°C[mΩ]_Score,Y POS1 R 75°C[mΩ]_Score,Y POS1 S 75°C[mΩ]_Score,Y POS1 T 75°C[mΩ]_Score,ROHM
0,146916,2019-06-30,149.785000,147.781000,144.857000,142.047000,140.025000,137.230000,134.387000,132.398000,...,1,1,1,1,1,1,NaN,NaN,NaN,1.0
1,146916,2015-01-06,148.965511,147.057225,144.194798,141.451638,139.424085,136.800193,133.937765,132.148748,...,1,1,1,5,5,5,NaN,NaN,NaN,5.0
2,146916,2014-11-12,144.028680,142.225767,139.389184,136.732893,134.761709,132.081379,129.340951,127.417845,...,1,1,1,1,1,1,NaN,NaN,NaN,1.0
3,146917,2022-05-12,153.255000,150.592000,147.916000,145.976000,143.261000,140.604000,138.741000,136.053000,...,1,1,1,5,5,5,NaN,NaN,NaN,5.0
4,146917,2014-12-05,157.346194,154.665864,151.913417,149.942233,147.153728,144.449359,142.514233,139.785825,...,1,1,1,1,1,1,NaN,NaN,NaN,1.0


In [ ]:
df_ROHM = df[['SERIE','FECHA','ROHM']]
df_ROHM.head()

,SERIE,FECHA,ROHM
0,146916,2019-06-30,1.0
1,146916,2015-01-06,5.0
2,146916,2014-11-12,1.0
3,146917,2022-05-12,5.0
4,146917,2014-12-05,1.0
